
# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [116]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

In [117]:
df = pd.read_csv('previsao_de_renda.csv')

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [119]:
# excluir valores nulos sem sentido
df = df.dropna()
# excluir colunas desnecessárias. 
df = df.drop(['Unnamed: 0','data_ref','id_cliente'], axis=1)


In [120]:
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [121]:
# 1)
df_treino, df_teste = train_test_split(df, test_size=0.25, random_state=100)

In [103]:
# 2)
modelo = 'renda ~ sexo + posse_de_veiculo + C(posse_de_imovel, Treatment(1)) + qtd_filhos + C(tipo_renda, Treatment(0)) + C(educacao, Treatment(2)) + estado_civil + C(tipo_residencia, Treatment(1)) + idade+ tempo_emprego+ qt_pessoas_residencia'

alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

for i in alpha:
    md = smf.ols(modelo, data = df_teste)
    reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.00001
                         , alpha = i)
    
    reg_summary = reg.summary()
    print(reg_summary)

                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     53.80
Date:                Sun, 25 Aug 2024   Prob (F-statistic):          1.76e-220
Time:                        12:03:49   Log-Likelihood:                -31733.
No. Observations:                3107   AIC:                         6.352e+04
Df Residuals:                    3082   BIC:                         6.367e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     53.80
Date:                Sun, 25 Aug 2024   Prob (F-statistic):          1.76e-220
Time:                        12:03:51   Log-Likelihood:                -31733.
No. Observations:                3107   AIC:                         6.352e+04
Df Residuals:                    3082   BIC:                         6.367e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     53.80
Date:                Sun, 25 Aug 2024   Prob (F-statistic):          1.76e-220
Time:                        12:03:53   Log-Likelihood:                -31733.
No. Observations:                3107   AIC:                         6.352e+04
Df Residuals:                    3082   BIC:                         6.367e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

In [104]:
modelo = 'renda ~ sexo + posse_de_veiculo + C(posse_de_imovel, Treatment(1)) + qtd_filhos + C(tipo_renda, Treatment(0)) + C(educacao, Treatment(2)) + estado_civil + C(tipo_residencia, Treatment(1)) + idade+ tempo_emprego+ qt_pessoas_residencia'

alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

for i in alpha:
    md = smf.ols(modelo, data = df_teste)
    reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.00001
                         , alpha = i)
    
    print(f'R² = {reg.rsquared} Adj. R² = {reg.rsquared_adj} Alpha = {i} ')

R² = 0.30380902063807236 Adj. R² = 0.2983876762173435 Alpha = 0 
R² = 0.30380902063807236 Adj. R² = 0.2983876762173435 Alpha = 0.001 
R² = 0.30380902063807236 Adj. R² = 0.2983876762173435 Alpha = 0.005 
R² = 0.30380902063807236 Adj. R² = 0.2983876762173435 Alpha = 0.01 
R² = 0.30380902063807236 Adj. R² = 0.2983876762173435 Alpha = 0.05 
R² = 0.30380902063807236 Adj. R² = 0.2983876762173435 Alpha = 0.1 


Observamos que todos os modelos ficaram iguais, variando pouco o R² , R² ajustado e o AIC.

In [105]:
# 3)
modelo = 'renda ~ sexo + posse_de_veiculo + C(posse_de_imovel, Treatment(1)) + qtd_filhos + C(tipo_renda, Treatment(0)) + C(educacao, Treatment(2)) + estado_civil + C(tipo_residencia, Treatment(1)) + idade+ tempo_emprego+ qt_pessoas_residencia'

alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

for i in alpha:
    md = smf.ols(modelo, data = df_teste)
    reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = i)
    
    reg_summary = reg.summary()
    print(reg_summary)

                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     53.80
Date:                Sun, 25 Aug 2024   Prob (F-statistic):          1.76e-220
Time:                        12:04:14   Log-Likelihood:                -31733.
No. Observations:                3107   AIC:                         6.352e+04
Df Residuals:                    3082   BIC:                         6.367e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     53.80
Date:                Sun, 25 Aug 2024   Prob (F-statistic):          1.76e-220
Time:                        12:04:16   Log-Likelihood:                -31733.
No. Observations:                3107   AIC:                         6.352e+04
Df Residuals:                    3082   BIC:                         6.367e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     53.80
Date:                Sun, 25 Aug 2024   Prob (F-statistic):          1.76e-220
Time:                        12:04:18   Log-Likelihood:                -31733.
No. Observations:                3107   AIC:                         6.352e+04
Df Residuals:                    3082   BIC:                         6.367e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------

In [106]:
modelo = 'renda ~ sexo + posse_de_veiculo + C(posse_de_imovel, Treatment(1)) + qtd_filhos + C(tipo_renda, Treatment(0)) + C(educacao, Treatment(2)) + estado_civil + C(tipo_residencia, Treatment(1)) + idade+ tempo_emprego+ qt_pessoas_residencia'

alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

for i in alpha:
    md = smf.ols(modelo, data = df_teste)
    reg = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = i)
    
    print(f'R² = {reg.rsquared} /// Adj. R² = {reg.rsquared_adj} Alpha = {i} ')

R² = 0.30380902063807236 /// Adj. R² = 0.2983876762173435 Alpha = 0 
R² = 0.30380902063807236 /// Adj. R² = 0.2983876762173435 Alpha = 0.001 
R² = 0.30380902063807236 /// Adj. R² = 0.2983876762173435 Alpha = 0.005 
R² = 0.30380902063807236 /// Adj. R² = 0.2983876762173435 Alpha = 0.01 
R² = 0.30380902063807236 /// Adj. R² = 0.2983876762173435 Alpha = 0.05 
R² = 0.30380902063807236 /// Adj. R² = 0.2983876762173435 Alpha = 0.1 


O resultado melhor foi com o Alpha = 0 e Alpha = 0,001

In [122]:
# 4)

#X_step = df_teste.drop('renda', axis = 1)
X_step = pd.get_dummies(df_teste, columns = ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], 
                        drop_first=True)

y_step = df_teste.renda
#y_step = df_teste.target


In [123]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [124]:
# devemos converter os dados para numérico
X_step = X_step.apply(pd.to_numeric, errors='coerce')

X_step = X_step.astype(int)


In [125]:
selected_features = stepwise_selection(X_step, y_step)

print("Selected features:", selected_features)

Add  renda                          with p-value 0.0
#############
['renda']
#############
['renda']
Selected features: ['renda']
